# We need to get access token.

In [ ]:
# https://stackapps.com/questions/9345/how-can-i-quickly-get-an-access-token-for-personal-use    #  scope = no_expiry,private_info
# Client Id   27047
# Key     Olal2qHePkJ7XZjbiuZi6Q((
# Client Secret  A5iQJXm19vCrKMCq2*z57A((

# our no-expiry access-token  zLR4yModLSN4nIvBcaTwWw))

In [ ]:
# an exaple to  stckapp tool to set filters: https://api.stackexchange.com/docs/questions#page=1&pagesize=110&order=desc&min=1&sort=votes&filter=!0_VI1fTYGwbDM-UWZMWbHaM5TzYg4j059lF8XZiZU&site=judaism&run=true

In [ ]:
# every page has 100 questions , max paging is 100  = 10k , site has 36k questions , correct to August 2023

# different from date to date 2010-2015   1262304000 1451606399
# different from date to date 2016-2018   fromdate=1451606400&todate=1546214400
# different from date to date 2019-2020   1546214401 1609459199
# different from date to date 2021        1609459200

In [3]:
!pip install -Uqq ipdb
import ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00


# Code

In [5]:
import requests
import html
import json

question_answer_list = []

url = "https://api.stackexchange.com/2.3/questions"
params = {
    "page": 1,
    "pagesize": 100,
    "order": "desc",
    "min": 1,
    "sort": "votes",
    "site": "judaism",
    "filter": "!7uBGi3F7bE36rVSYESQ.7L1AF9jnxYd.GVN0XFwR3c6NUI", #"!0_VI1fTYGwbDM-UWZMWbHaM5TzYg4j059lF8XZiZU",
    "access_token": "zLR4yModLSN4nIvBcaTwWw))",
    "key": "Olal2qHePkJ7XZjbiuZi6Q(("
}

no_answer = 0
PTIJ = 0

a = ['1262304000','1451606400','1546214401','1609459200']
b = ['1451606399','1546214400','1609459199','']

for from_date, to_date in zip(a, b):
    params["fromdate"] = from_date
    if to_date:
        params["todate"] = to_date
    else:
        del params["todate"]
    params["page"] = 1

    while params["page"] < params["pagesize"] +1:
      data_list = []

      response = requests.get(url, params=params)
      response_data = response.json()

      if "items" in response_data:
          data_list.extend(response_data["items"])

      has_more = response_data.get("has_more", False)


      params["page"] += 1

      for data in data_list:
        if data['body_markdown'].startswith("PTIJ:"): # purim-torah-in-jest, funny questions for purim, not relevant to us
            PTIJ = PTIJ + 1
            continue
        if "answers" not in data:
            no_answer = no_answer + 1
            continue

        question = html.unescape(data["body_markdown"]) if "body_markdown" in data else ""
        link = html.unescape(data["link"]) if "link" in data else ""
        title = html.unescape(data["title"]) if "title" in data else ""

        data = data['answers']
        highest_score_entry = next((entry for entry in data if entry['accepted']), None)

        # If no accepted answers are found, fall back to the first entry with a score above 2
        if highest_score_entry is None:
            highest_score_entry = data[0]  if data[0]['score'] > 1 else None
        if highest_score_entry is None:
            no_answer = no_answer + 1
            continue
        else:
          accepted_markdowns = highest_score_entry['body_markdown']
          accepted_markdowns = html.unescape(accepted_markdowns)

        answer = accepted_markdowns

        # Create a dictionary for each question and answer pair
        qa_pair = {
            "question": question,
            "answer": answer,
            "title": title,
            "link": link
        }
        question_answer_list.append(qa_pair)

      if not has_more:
          break

json_data = json.dumps(question_answer_list, indent=4)

# Save the JSON to a file
with open("question_answer_data.json", "w") as json_file:
    json_file.write(json_data)

print('no answers: ' + str(no_answer))
print('PTIJ: ' + str(PTIJ))

no answers: 11193
PTIJ: 0
